# Samba Insight - Exploratory Data Analysis

**Project:** Brazilian E-Commerce Analytics

**Purpose:** Exploratory analysis of BigQuery warehouse data to uncover insights and patterns.

**Data Sources:**
- `fact_orders` - Order-level transactions
- `mart_sales_monthly` - Monthly sales aggregations
- `mart_customer_cohorts` - Customer cohort analysis
- `mart_product_performance` - Product category metrics
- `mart_sales_daily` - Daily sales by geography
- `mart_customer_retention` - Retention analysis

---

## Setup

Import libraries and initialize notebook environment.

In [ ]:
# Import libraries
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Add current directory to path for importing notebook_utils
notebook_dir = Path().absolute()
if str(notebook_dir) not in sys.path:
    sys.path.insert(0, str(notebook_dir))

# Import utilities
from notebook_utils import (
    DataValidator,
    VisualizationHelper,
    export_dataframe,
    setup_notebook_environment,
)

# Setup environment
config, client, bq_helper = setup_notebook_environment()
viz = VisualizationHelper()

print("✅ Environment ready for analysis")

## 1. Business Overview

Load key business metrics to understand overall performance.

In [ ]:
# Query: Overall business metrics
query_overview = f"""
SELECT
  COUNT(DISTINCT order_id) as total_orders,
  COUNT(DISTINCT customer_key) as total_customers,
  ROUND(SUM(total_order_value), 2) as total_revenue,
  ROUND(AVG(total_order_value), 2) as avg_order_value,
  ROUND(AVG(review_score), 2) as avg_review_score,
  MIN(order_purchase_date) as first_order_date,
  MAX(order_purchase_date) as last_order_date,
  DATE_DIFF(MAX(order_purchase_date), MIN(order_purchase_date), DAY) as days_of_data
FROM `{config.project_id}.{config.dataset}.fact_orders`
WHERE order_status = 'delivered'
"""

# Execute query
df_overview = bq_helper.query_to_dataframe(query_overview)
DataValidator.check_dataframe_not_empty(df_overview, "Business Overview")

# Display results
print("\n📊 Business Overview")
print("=" * 70)
for col in df_overview.columns:
    print(f"{col:.<50} {df_overview[col].iloc[0]}")
print("=" * 70)

## 2. Revenue Trends

Analyze revenue and order volume trends over time.

In [ ]:
# Query: Monthly revenue trends
query_revenue = f"""
SELECT
  FORMAT_DATE('%Y-%m', month_date) as month,
  SUM(total_revenue) as revenue,
  SUM(total_orders) as orders,
  AVG(avg_order_value) as aov
FROM `{config.project_id}.{config.dataset}.mart_sales_monthly`
GROUP BY month_date, month
ORDER BY month_date
"""

# Execute query
df_revenue = bq_helper.query_to_dataframe(query_revenue)
DataValidator.check_dataframe_not_empty(df_revenue, "Revenue Data")

# Create visualizations
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Revenue trend
ax1.plot(
    df_revenue["month"], df_revenue["revenue"], marker="o", linewidth=2, color=viz.colors["primary"]
)
ax1.set_title("Monthly Revenue Trend", fontsize=16, fontweight="bold")
ax1.set_xlabel("Month", fontsize=12)
ax1.set_ylabel("Revenue (R$)", fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis="x", rotation=45)

# Order volume trend
ax2.bar(df_revenue["month"], df_revenue["orders"], color=viz.colors["success"], alpha=0.7)
ax2.set_title("Monthly Order Volume", fontsize=16, fontweight="bold")
ax2.set_xlabel("Month", fontsize=12)
ax2.set_ylabel("Orders", fontsize=12)
ax2.grid(True, alpha=0.3, axis="y")
ax2.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

# Calculate growth
if len(df_revenue) > 1:
    first_revenue = df_revenue["revenue"].iloc[0]
    last_revenue = df_revenue["revenue"].iloc[-1]
    growth_pct = ((last_revenue / first_revenue) - 1) * 100
    print(f"\n💡 Revenue grew from R${first_revenue:,.0f} to R${last_revenue:,.0f}")
    print(f"💡 Total growth: {growth_pct:.1f}%")

## 3. Customer Segmentation

Analyze customer segments and their value contribution.

In [ ]:
# Query: Customer segments
query_segments = f"""
SELECT
  customer_segment,
  COUNT(*) as customers,
  AVG(total_orders) as avg_orders,
  AVG(total_orders * 150) as estimated_ltv,
  AVG(avg_review_score) as avg_rating
FROM `{config.project_id}.{config.dataset}.dim_customer`
WHERE total_orders > 0
GROUP BY customer_segment
ORDER BY customers DESC
"""

# Execute query
df_segments = bq_helper.query_to_dataframe(query_segments)
DataValidator.check_dataframe_not_empty(df_segments, "Customer Segments")

# Create visualizations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Segment distribution pie chart
segment_colors = [viz.colors["warning"], viz.colors["success"], viz.colors["danger"]]
ax1.pie(
    df_segments["customers"],
    labels=df_segments["customer_segment"],
    autopct="%1.1f%%",
    colors=segment_colors,
    startangle=90,
)
ax1.set_title("Customer Segment Distribution", fontsize=14, fontweight="bold")

# LTV by segment bar chart
ax2.bar(
    df_segments["customer_segment"], df_segments["estimated_ltv"], color=segment_colors, alpha=0.7
)
ax2.set_title("Estimated Lifetime Value by Segment", fontsize=14, fontweight="bold")
ax2.set_ylabel("LTV (R$)", fontsize=12)
ax2.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

# Display summary
print("\n📊 Customer Segment Summary")
print(df_segments.to_string(index=False))

## 4. Product Performance

Identify top-performing product categories.

In [ ]:
# Query: Top product categories
query_categories = f"""
SELECT
  product_category,
  SUM(total_orders) as orders,
  SUM(total_revenue) as revenue,
  AVG(avg_review_score) as avg_rating
FROM `{config.project_id}.{config.dataset}.mart_product_performance`
WHERE product_category IS NOT NULL
GROUP BY product_category
ORDER BY revenue DESC
LIMIT 15
"""

# Execute query
df_categories = bq_helper.query_to_dataframe(query_categories)
DataValidator.check_dataframe_not_empty(df_categories, "Product Categories")

# Create horizontal bar chart
fig, ax = plt.subplots(figsize=(12, 8))

y_pos = np.arange(len(df_categories))
ax.barh(y_pos, df_categories["revenue"], color=viz.colors["primary"], alpha=0.7)
ax.set_yticks(y_pos)
ax.set_yticklabels(df_categories["product_category"])
ax.invert_yaxis()
ax.set_xlabel("Revenue (R$)", fontsize=12)
ax.set_title("Top 15 Product Categories by Revenue", fontsize=16, fontweight="bold")
ax.grid(True, alpha=0.3, axis="x")

plt.tight_layout()
plt.show()

# Display top category
print(f"\n🏆 Top Category: {df_categories['product_category'].iloc[0]}")
print(f"   Revenue: R${df_categories['revenue'].iloc[0]:,.0f}")

## 5. Geographic Distribution

Analyze sales distribution across Brazilian states.

In [ ]:
# Query: Sales by state
query_geo = f"""
SELECT
  customer_state,
  SUM(total_orders) as orders,
  SUM(total_revenue) as revenue,
  COUNT(DISTINCT customer_city) as cities
FROM `{config.project_id}.{config.dataset}.mart_sales_daily`
GROUP BY customer_state
ORDER BY revenue DESC
LIMIT 10
"""

# Execute query
df_geo = bq_helper.query_to_dataframe(query_geo)
DataValidator.check_dataframe_not_empty(df_geo, "Geographic Data")

# Create dual-axis chart
fig, ax1 = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_geo))
width = 0.35

# Orders bar chart
ax1.bar(
    x - width / 2, df_geo["orders"], width, label="Orders", color=viz.colors["success"], alpha=0.7
)
ax1.set_xlabel("State", fontsize=12)
ax1.set_ylabel("Orders", fontsize=12, color=viz.colors["success"])
ax1.set_xticks(x)
ax1.set_xticklabels(df_geo["customer_state"])
ax1.tick_params(axis="y", labelcolor=viz.colors["success"])

# Revenue line chart
ax2 = ax1.twinx()
ax2.plot(x, df_geo["revenue"], marker="o", linewidth=2, color=viz.colors["danger"], label="Revenue")
ax2.set_ylabel("Revenue (R$)", fontsize=12, color=viz.colors["danger"])
ax2.tick_params(axis="y", labelcolor=viz.colors["danger"])

ax1.set_title("Top 10 States by Orders and Revenue", fontsize=16, fontweight="bold")
ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
ax1.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

# Display top states
print("\n🗺️  Top 5 States")
print(df_geo[["customer_state", "orders", "revenue"]].head().to_string(index=False))

## 6. Retention Analysis

Analyze customer retention patterns by cohort.

In [ ]:
# Query: Cohort retention
query_retention = f"""
SELECT
  cohort_month_name,
  months_since_cohort,
  retention_rate_pct,
  cohort_size
FROM `{config.project_id}.{config.dataset}.mart_customer_retention`
WHERE cohort_month >= '2017-01-01'
  AND months_since_cohort <= 12
ORDER BY cohort_month_name, months_since_cohort
"""

# Execute query
df_retention = bq_helper.query_to_dataframe(query_retention)
DataValidator.check_dataframe_not_empty(df_retention, "Retention Data")

# Pivot for heatmap
df_retention_agg = df_retention.groupby(
    ["cohort_month_name", "months_since_cohort"], as_index=False
).agg({"retention_rate_pct": "mean"})

pivot_retention = df_retention_agg.pivot(
    index="cohort_month_name", columns="months_since_cohort", values="retention_rate_pct"
)

# Create heatmap
fig, ax = viz.create_heatmap(
    pivot_retention,
    title="Customer Retention Heatmap by Cohort",
    cmap="RdYlGn",
    vmin=0,
    vmax=100,
    annot=True,
    fmt=".1f",
    cbar_kws={"label": "Retention Rate (%)"},
)
ax.set_xlabel("Months Since First Order", fontsize=12)
ax.set_ylabel("Cohort Month", fontsize=12)

plt.tight_layout()
plt.show()

# Display retention insights
print("\n💡 Retention Insights")
if 1 in pivot_retention.columns:
    print(f"  • Month 1 Retention: {pivot_retention[1].mean():.1f}%")
if 6 in pivot_retention.columns:
    print(f"  • Month 6 Retention: {pivot_retention[6].mean():.1f}%")
if 12 in pivot_retention.columns:
    print(f"  • Month 12 Retention: {pivot_retention[12].mean():.1f}%")

## 7. Key Insights

Automated insights generation based on the analysis.

In [ ]:
print("\n" + "=" * 80)
print("🎯 KEY INSIGHTS - SAMBA INSIGHT ANALYSIS")
print("=" * 80)

# Business Performance
print("\n📊 Business Performance")
print(f"  • Total Revenue: R${df_overview['total_revenue'].iloc[0]:,.0f}")
print(f"  • Total Orders: {df_overview['total_orders'].iloc[0]:,}")
print(f"  • Average Order Value: R${df_overview['avg_order_value'].iloc[0]:.2f}")
print(f"  • Customer Satisfaction: {df_overview['avg_review_score'].iloc[0]:.2f}/5.0 ⭐")

# Customer Insights
print("\n👥 Customer Insights")
print(f"  • Total Customers: {df_overview['total_customers'].iloc[0]:,}")
loyal_pct = (
    df_segments[df_segments["customer_segment"] == "loyal"]["customers"].iloc[0]
    / df_segments["customers"].sum()
    * 100
)
print(f"  • Loyal Customers: {loyal_pct:.1f}% of customer base")
print(
    f"  • Highest LTV Segment: {df_segments.loc[df_segments['estimated_ltv'].idxmax(), 'customer_segment']}"
)

# Product Insights
print("\n🏆 Product Insights")
print(f"  • Top Category: {df_categories['product_category'].iloc[0]}")
print(f"  • Top Category Revenue: R${df_categories['revenue'].iloc[0]:,.0f}")
top3_concentration = df_categories["revenue"].head(3).sum() / df_categories["revenue"].sum() * 100
print(f"  • Top 3 Concentration: {top3_concentration:.1f}% of total revenue")

# Geographic Insights
print("\n🗺️  Geographic Insights")
print(f"  • Top State: {df_geo['customer_state'].iloc[0]}")
print(f"  • Top State Revenue: R${df_geo['revenue'].iloc[0]:,.0f}")
geo_concentration = df_geo["revenue"].head(3).sum() / df_geo["revenue"].sum() * 100
print(f"  • Top 3 States: {geo_concentration:.1f}% of total revenue")

# Retention Insights
print("\n🔄 Retention Insights")
if "pivot_retention" in locals() and len(pivot_retention.columns) > 0:
    if 1 in pivot_retention.columns:
        print(f"  • 1-Month Retention: {pivot_retention[1].mean():.1f}%")
    if 6 in pivot_retention.columns:
        print(f"  • 6-Month Retention: {pivot_retention[6].mean():.1f}%")
    if 12 in pivot_retention.columns:
        print(f"  • 12-Month Retention: {pivot_retention[12].mean():.1f}%")

print("\n" + "=" * 80)
print("\n✅ Analysis Complete")
print(
    f"📅 Data Period: {df_overview['first_order_date'].iloc[0]} to {df_overview['last_order_date'].iloc[0]}"
)
print(f"📈 Total Days: {df_overview['days_of_data'].iloc[0]}")

## 8. Export Summary

Export key metrics for reporting and sharing.

In [ ]:
# Create summary DataFrame
summary_metrics = {
    "Metric": [
        "Total Revenue",
        "Total Orders",
        "Avg Order Value",
        "Total Customers",
        "Avg Review Score",
        "Top Product Category",
        "Top State",
    ],
    "Value": [
        f"R${df_overview['total_revenue'].iloc[0]:,.0f}",
        f"{df_overview['total_orders'].iloc[0]:,}",
        f"R${df_overview['avg_order_value'].iloc[0]:.2f}",
        f"{df_overview['total_customers'].iloc[0]:,}",
        f"{df_overview['avg_review_score'].iloc[0]:.2f}/5.0",
        df_categories["product_category"].iloc[0],
        df_geo["customer_state"].iloc[0],
    ],
}

df_summary = pd.DataFrame(summary_metrics)

# Export to multiple formats
exported_files = export_dataframe(
    df_summary, filename="eda_summary_metrics", formats=["csv", "xlsx"]
)

# Display summary
print("\n📊 Summary Metrics")
print(df_summary.to_string(index=False))